In [ ]:
# default_exp data

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Data

> This model contains all the necessary functionality for managing data.

In [ ]:
# export
import icodegen
import re

from pathlib import Path
from subprocess import check_output
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers, processors

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# hide
from ds4se.mgmnt.prep.i import jsonl_list_to_dataframe, get_dfs

path = Path('/home/nathan/Downloads/')
df_trn, df_val, df_tst = get_dfs(path/"java/final/jsonl")

sample = 0.01
df_trn = df_trn.sample(frac = sample)
df_val = df_val.sample(frac = sample)
df_tst = df_tst.sample(frac = sample)
df_trn.head()

,code,docstring
12678,@Override\n\tpublic void removeListeners(Strin...,--- REMOVE ALL REMOTE SERVICES/ACTIONS OF A NO...
665,public void loseWeight()\n {\n if (s...,去掉多余的buffer
22702,"public static JWT verify(String jwt, byte[] se...",Validate whether the JWT is valid. If valid re...
16110,public String updateByPrimaryKey(MappedStateme...,通过主键更新全部字段\n\n@param ms
853,public ArrayList<String> vps_serviceName_addit...,Get allowed durations for 'additionalDisk' opt...


In [ ]:
# hide
len(df_trn), len(df_val), len(df_tst)

(4545, 153, 269)

Uncrustify failure case. Will need to look into finding how many failure cases uncrustify generates.

Original:
```
@Override
    public WSStats[] getStatsArray(StatDescriptor[] sd, Boolean recursive) {
        if (tc.isEntryEnabled())
            Tr.entry(tc, "getStatsArray (StatDescriptor[], Boolean)");

        if (sd == null || sd.length == 0) {
            if (tc.isEntryEnabled())
                Tr.exit(tc, "StatDescriptor is null or empty");
            return null;
        }

        StatsImpl[] stats = PmiRegistry.getStats(sd, recursive.booleanValue(), new PmiModuleConfig(null));

        if (tc.isEntryEnabled())
            Tr.exit(tc, "getStatsArray (StatDescriptor[], Boolean)");

        return stats;
    }
```

Beautified:
```
@Override public WSStats[] getStatsArray(StatDescriptor[] sd, Boolean
    recursive) {
    if (tc.isEntryEnabled())
	Tr.entry(tc, "getStatsArray (StatDescriptor[], Boolean)");
    if (sd == null || sd.length == 0) {
	if (tc.isEntryEnabled())
```

In [ ]:
import pandas as pd

df_fake = pd.DataFrame([
    '''public void setPipelines(java.util.Collection<Pipeline> pipelines) {
        if (pipelines == null) {
            this.pipelines = null;
            return;
        }

        this.pipelines = new com.amazonaws.internal.SdkInternalList<Pipeline>(pipelines);
    }
    '''
], columns = ['code']); df_fake

,code
0,public void setPipelines(java.util.Collection<...


In [ ]:
# export
def _beautify(mthd):
    # get path of icodegen
    icodegen_path = Path(icodegen.__path__[0])

    # create tmp file to store df contents for training tokenizer
    tmp_path = Path('/tmp')
    tmp_path.mkdir(parents = True, exist_ok = True)
    with open(tmp_path/'tmp.java', 'w') as f:
        f.write(mthd)

    beaut_mthd = check_output([
        icodegen_path/'uncrustify', '-c', icodegen_path/'sun.cfg',
        '-f', tmp_path/'tmp.java'
    ]).decode('utf-8')

    return beaut_mthd

# TODO: fix bugs that cause to crash/hang on certain inputs
def beautify_code(df, n = None):
    """
    Beautify the methods in a pandas dataframe using uncrustify with sun.cfg style, i.e., Oracle's style.

    :param df: the pandas dataframe containing each method to be beautified
    :param n: the number of methods to evaluate. If none, the entire dataframe will be used
    :returns: returns a modified dataframe with the methods beautified
    """
    if n is None: n = len(df)

    df = df.iloc[:n].copy()
    df.code = df.code.apply(lambda mthd: _beautify(mthd))
    
    return df

In [ ]:
BEAUT_MTHD = '''public void setPipelines(java.util.Collection<Pipeline> pipelines) {
    if (pipelines == null) {
	this.pipelines = null;
	return;
    }
    this.pipelines = new com.amazonaws.internal.SdkInternalList<Pipeline>(
	pipelines);
}
'''

df_beaut = beautify_code(df_fake)

assert BEAUT_MTHD == df_beaut.code.values[0]

In [ ]:
# hide
%time df_beaut = beautify_code(df_trn, n = 10)

CPU times: user 14.2 ms, sys: 76.3 ms, total: 90.6 ms
Wall time: 273 ms


In [ ]:
# hide
idx = 0
print(df_trn.code.values[idx])
print(df_beaut.code.values[idx])

@Override
	public void removeListeners(String nodeID) {
		boolean found = false;
		writeLock.lock();
		try {
			Iterator&lt;HashMap&lt;String, Strategy&lt;ListenerEndpoint&gt;&gt;&gt; groupIterator = listeners.values().iterator();
			while (groupIterator.hasNext()) {
				HashMap&lt;String, Strategy&lt;ListenerEndpoint&gt;&gt; groups = groupIterator.next();
				Iterator&lt;Strategy&lt;ListenerEndpoint&gt;&gt; strategyIterator = groups.values().iterator();
				while (strategyIterator.hasNext()) {
					Strategy&lt;ListenerEndpoint&gt; strategy = strategyIterator.next();
					if (strategy.remove(nodeID)) {
						found = true;
						if (strategy.isEmpty()) {
							strategyIterator.remove();
						}
					}
				}
				if (groups.isEmpty()) {
					groupIterator.remove();
				}
			}
		} finally {

			// Clear caches
			if (found) {
				emitterCache.clear();
				broadcasterCache.clear();
				localBroadcasterCache.clear();
			}

			writeLock.unlock();
		}
	}
@Override public void removeListeners(

In [ ]:
# export
# dicts of special tokens we are adding to the tokenizers so they do not get split

# from https://docs.oracle.com/javase/tutorial/java/nutsandbolts/_keywords.html
java_reserved_tokens = {
    'abstract': '<abstract>', 'assert': '<assert>', 'boolean': '<boolean>',
    'break': '<break>', 'byte': '<byte>', 'case': '<case>',
    'catch': '<catch>', 'char': '<char>', 'class': '<class>',
    'const': '<const>', 'continue': '<continue>', 'default': '<default>',
    'do': '<do>', 'double': '<double>', 'else': '<else>',
    'enum': '<enum>', 'extends': '<extends>', 'final': '<final>',
    'finally': '<finally>', 'float': '<float>', 'for': '<for>',
    'goto': '<goto>', 'if': '<if>', 'implements': '<implements>',
    'import': '<import>', 'instanceof': '<instanceof>', 'int': '<int>',
    'interface': '<interface>', 'long': '<long>', 'native': '<native>',
    'new': '<new>', 'package': '<package>', 'private': '<private>',
    'protected': '<protected>', 'public': '<public>', 'return': '<return>',
    'short': '<short>', 'static': '<static>', 'strictfp': '<strictfp>',
    'super': '<super>', 'switch': '<switch>', 'synchronized': '<synchronized>',
    'this': '<this>', 'throw': '<throw>', 'throws': '<throws>',
    'transient': '<transient>', 'try': '<try>', 'void': '<void>',
    'volatile': '<volatile>', 'while': '<while>'
}

# from https://docs.oracle.com/javase/tutorial/java/nutsandbolts/opsummary.html
java_operator_tokens = {
    '=': '<=>', '+': '<+>', '-': '<->',
    '*': '<*>', '/': '</>', '%': '<%>',
    '++': '<++>', '--': '<-->', '!': '<!>',
    '==': '<==>', '!=': '<!=>', '>': '<greater>',
    '>=': '<greater_equal>', '<': '<lesser>', '<=': '<lesser_equal>',
    '&&': '<&&>', '||': '<||>', '?': '<?>',
    ':': '<:>', '~': '<~>', '<<': '<double_lesser>',
    '>>': '<double_greater>', '>>>': '<triple_greater>', '&': '<&>',
    '^': '<^>', '|': '<|>'
}

java_structural_tokens = {
    '{': '<{>', '}': '<}>', '[': '<[>',
    ']': '<]>', '<': '<lesser>', '>': '<greater>',
    '(': '<(>', ')': '<)>', ';': '<;>'
}

java_extra_tokens = {
    '@': '<@>', '...': '<...>'
}

extra_tokens = {
    ' ': '<space>', '\n': '<n>', '\t': '<t>'
}

# combination of all dictionaries
java_special_tokens = {
    **java_reserved_tokens, **java_operator_tokens, **java_structural_tokens,
    **java_extra_tokens, **extra_tokens
}

In [ ]:
df_fake = pd.DataFrame(['>>> > + public ++ \n\n \t \t \t\t  '], columns = ['code']); df_fake

,code
0,>>> > + public ++ \n\n \t \t \t\t


In [ ]:
# export
def _replace_toks(mthd, spec_toks):
    """
    Helper function for replacing all special tokens in a given method. This will replace longer special tokens first in order to not mistakenly breakup a special token that is part of a longer sequence. Adapted from https://stackoverflow.com/a/6117124/5768407 and https://stackoverflow.com/a/11753945/5768407

    :param mthd: the method to have it's special tokens replaced
    :param spec_toks: a dictionary containing the special tokens to replace and the new tokens to replace them with
    :returns: returns the method with all of its special tokens replaced
    """
    # construct escaped versions of keys for running through regex
    spec_toks = dict((re.escape(k), spec_toks[k]) for k in sorted(spec_toks, key=len, reverse=True))
    # construct regex pattern for finding all special tokens in a method
    pattern = re.compile("|".join(spec_toks.keys()))
    # replace all special tokens in a method
    mthd = pattern.sub(lambda m: spec_toks[re.escape(m.group(0))], mthd)

    return mthd

def replace_special_tokens(df, spec_toks, n = None):
    """
    Replace all the special tokens in a pandas dataframe.

    :param df: the pandas dataframe containing each method to replace special tokens in
    :param n: the number of methods to evaluate. If none, the entire dataframe will be used
    :returns: returns a modified dataframe with the special tokens replaced
    """
    if n is None: n = len(df)

    df = df.iloc[:n].copy()
    df.code = df.code.apply(lambda mthd: _replace_toks(mthd, spec_toks))

    return df

In [ ]:
REPLACED_MTHD = '<triple_greater><space><greater><space><+><space><public><space><++><space><n><n><space><t><space><t><space><t><t><space><space>'
df_replaced = replace_special_tokens(df_fake, java_special_tokens)

assert REPLACED_MTHD == df_replaced.code.values[0]

In [ ]:
# hide
df_replaced = replace_special_tokens(df_trn, java_special_tokens)
print(df_replaced.code.values[5])

&lt;private&gt;&lt;space&gt;&lt;void&gt;&lt;space&gt;setFKField&lt;(&gt;Object&lt;space&gt;targetObject,&lt;space&gt;ClassDescriptor&lt;space&gt;cld,&lt;space&gt;ObjectReferenceDescriptor&lt;space&gt;rds,&lt;space&gt;Object&lt;space&gt;referencedObject&lt;)&gt;&lt;n&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;{&gt;&lt;n&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;ValueContainer&lt;[&gt;&lt;]&gt;&lt;space&gt;refPkValues&lt;;&gt;&lt;n&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;FieldDescriptor&lt;space&gt;fld&lt;;&gt;&lt;n&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;FieldDescriptor&lt;[&gt;&lt;]&gt;&lt;space&gt;objFkFields&lt;space&gt;&lt;=&gt;&lt;space&gt;rds.getForeignKeyFieldDescriptors&lt;(&gt;cld&lt;)&gt;&lt;;&gt;&lt;n&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space

In [ ]:
fake_data = '<triple_greater><space><greater><space><+><space><public><space><++><space><n><n><space><t><space><t><space><t><t><space><space>'

In [ ]:
# export
# TODO: test case and stop merges across camel and snake case
def train_tokenizer(df, n = None, vocab_sz = 20_000, min_freq = 2, output = None):
    """
    Train a ByteLevel BPE tokenizer on a given pandas dataframe. Code adapted from https://github.com/huggingface/tokenizers/tree/master/bindings/python.

    :param df: the pandas dataframe containing each method to have the tokenizer train on
    :param n: the number of methods to evaluate. If none, the entire dataframe will be used
    :param vocab_sz: the maximum vocabulary size of the trained tokenizer
    :param min_freq: the minimum frequency a token has to occur to be considered
    :returns: returns a trained ByteLevel BPE tokenizer
    """
    if n is None: n = len(df)

    # create tmp file to store df contents for training tokenizer
    tmp_path = Path('/tmp')
    tmp_path.mkdir(parents = True, exist_ok = True)
    with open(tmp_path/'tmp_tokenize.txt', 'w') as f:
        f.write('\n'.join(df.code.values[:n]))

    # initialize a tokenizer
    tokenizer = Tokenizer(models.BPE())

    # customize pre-tokenization and decoding
    tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space = True)
    tokenizer.decoder = decoders.ByteLevel()
    tokenizer.post_processor = processors.ByteLevel(trim_offsets = True)

    # train tokenizer with data in tmp file
    trainer = trainers.BpeTrainer(
        vocab_size = vocab_sz, min_frequency = min_freq,
        special_tokens = list(java_special_tokens.values())
    )
    tokenizer.train(trainer, [str(tmp_path/'tmp_tokenize.txt')])

    # save tokenizer if output path given
    if output is not None:
        tokenizer.save(output, pretty = True)
    
    return tokenizer

In [ ]:
tokenizer = train_tokenizer(df_fake)

In [ ]:
TOKENIZED_SPEC = [
    '<triple_greater>', '<space>', '<greater>', '<space>', '<+>', '<space>',
    '<public>', '<space>', '<++>', '<space>', '<n>', '<n>',
    '<space>', '<t>', '<space>', '<t>', '<space>', '<t>',
    '<t>', '<space>', '<space>'
]

encoded = tokenizer.encode(fake_data)

assert TOKENIZED_SPEC == encoded.tokens

In [ ]:
# TODO: add test case for more realistic example, i.e., real method. Issue is generating small fake data to train tokenizer on. Are these tokenizers deterministic when training?

In [ ]:
# hide
idx = 0
df_beaut = beautify_code(df_trn, n = 10)
df_replaced = replace_special_tokens(df_beaut, java_special_tokens)

tokenizer = train_tokenizer(df_trn)
encoded = tokenizer.encode(df_replaced.code.values[idx])
print(df_replaced.code.values[idx])
print('=' * 100)
print(encoded.tokens)

&lt;@&gt;Override&lt;space&gt;&lt;public&gt;&lt;space&gt;&lt;void&gt;&lt;space&gt;removeListeners&lt;(&gt;String&lt;space&gt;nodeID&lt;)&gt;&lt;space&gt;&lt;{&gt;&lt;n&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;boolean&gt;&lt;space&gt;found&lt;space&gt;&lt;=&gt;&lt;space&gt;false&lt;;&gt;&lt;n&gt;&lt;n&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;writeLock.lock&lt;(&gt;&lt;)&gt;&lt;;&gt;&lt;n&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;try&gt;&lt;space&gt;&lt;{&gt;&lt;n&gt;&lt;t&gt;Iterator&lt;lesser&gt;HashMap&lt;lesser&gt;String,&lt;space&gt;Strategy&lt;lesser&gt;ListenerEndpo&lt;int&gt;&lt;greater&gt;&lt;space&gt;&lt;greater&gt;&lt;space&gt;&lt;greater&gt;&lt;space&gt;groupIterator&lt;space&gt;&lt;=&gt;&lt;n&gt;&lt;t&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;&lt;space&gt;listeners.values&lt;(&gt;&lt;)&gt;.iterator&lt;(&gt;&lt;)&gt;&lt;;&gt;&lt;n&gt;&lt;n&gt;&lt;t&gt;&lt;while&gt;&lt;space&gt;&lt;(&gt;groupIterator.hasNext&lt;(&gt;&lt;)&gt;&lt